## **LCEL integration**

In [26]:
from langchain_mistralai import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import getpass
import os
from dotenv import load_dotenv

load_dotenv()

True

In [27]:
def get_mistral_api_key():
    key = os.getenv("MISTRAL_API_KEY")
    if not key:
        key = getpass.getpass("Enter your Mistral API key: ")
        os.environ["MISTRAL_API_KEY"] = key
    return key

# Get the Mistral API key
mistral_api_key = get_mistral_api_key()

In [28]:
prompt = ChatPromptTemplate.from_template(
    """
    Yuo are a helpful assistant.
    Answer the folowwing questions: {question}
    """
)

In [29]:
llm = ChatMistralAI(model="mistral-small-2501",
    temperature=0,
    max_retries=2,)

In [30]:
output_parser = StrOutputParser()

In [31]:
chain = prompt | llm | output_parser

In [32]:
chain.invoke({"question":"Tell me about The Godfather movie"})

'"The Godfather" is a classic American crime film directed by Francis Ford Coppola, released in 1972. It is widely regarded as one of the greatest films ever made and is based on Mario Puzo\'s best-selling novel of the same name. The film stars Marlon Brando, Al Pacino, James Caan, Robert Duvall, and Diane Keaton, among others.\n\n### Plot Overview\nThe story revolves around the Corleone family, a powerful Mafia dynasty in New York City. The film begins with the wedding of Connie Corleone (Talia Shire), the youngest daughter of Don Vito Corleone (Marlon Brando), the head of the family. The wedding is a pivotal moment where various characters are introduced, including Vito\'s sons: Sonny (James Caan), Fredo (John Cazale), and Michael (Al Pacino).\n\nThe narrative then shifts to the attempted assassination of Vito Corleone, which leads to Michael, initially reluctant to join the family business, taking over the reins of the family after his father is wounded. Michael\'s transformation fr

### Check the imput schema

In [33]:
chain.input_schema.model_json_schema()

{'properties': {'question': {'title': 'Question', 'type': 'string'}},
 'required': ['question'],
 'title': 'PromptInput',
 'type': 'object'}

In [34]:
chain.output_schema.model_json_schema()

{'title': 'StrOutputParserOutput', 'type': 'string'}

In [35]:
llm.input_schema.model_json_schema()

{'$defs': {'AIMessage': {'additionalProperties': True,
   'description': 'Message from an AI.\n\nAIMessage is returned from a chat model as a response to a prompt.\n\nThis message represents the output of the model and consists of both\nthe raw output as returned by the model together standardized fields\n(e.g., tool calls, usage metadata) added by the LangChain framework.',
   'properties': {'content': {'anyOf': [{'type': 'string'},
      {'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]},
       'type': 'array'}],
     'title': 'Content'},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'response_metadata': {'title': 'Response Metadata', 'type': 'object'},
    'type': {'const': 'ai',
     'default': 'ai',
     'title': 'Type',
     'type': 'string'},
    'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'title': 'Name'},
    'id': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     '

### Implement Streaming

In [36]:
for chunk in chain.stream({"question":"Tell me about The Godfather movie"}):
    print(chunk)


"The
 God
father
"
 is
 a
 classic
 American
 crime
 film
 directed
 by
 Francis
 Ford
 Copp
ola
,
 released
 in
 
1
9
7
2
.
 It
 is
 widely
 regarded
 as
 one
 of
 the
 greatest
 films
 ever
 made
 and
 is
 based
 on
 Mario
 Pu
zo
's
 best
-selling
 novel
 of
 the
 same
 name
.
 The
 film
 stars
 Mar
lon
 Br
ando
,
 Al
 Pac
ino
,
 James
 C
aan
,
 Robert
 D
uv
all
,
 and
 Diane
 Ke
aton
,
 among
 others
.


###
 Plot
 Overview


The
 story
 revol
ves
 around
 the
 Cor
le
one
 family
,
 a
 powerful
 Mafia
 dynasty
 in
 New
 York
 City
.
 The
 film
 begins
 with
 the
 wedding
 of
 Connie
 Cor
le
one
 (
T
alia
 Sh
ire
),
 the
 youngest
 daughter
 of
 Don
 Vito
 Cor
le
one
 (
M
arl
on
 Br
ando
),
 the
 head
 of
 the
 family
.
 The
 wedding
 is
 a
 pivotal
 moment
 where
 various
 characters
 are
 introduced
,
 including
 Vito
's
 sons
:
 Sonny
 (
James
 C
aan
),
 Fred
o
 (
John
 Caz
ale
),
 and
 Michael
 (
Al
 Pac
ino
).


The
 narrative
 then
 shifts
 to
 the
 attempted
 assassination
 o

### Use batch technick

In [37]:
questions = [
    {"question":"Tell me about The Godfather movie"},
    {"question":"Tell me about The Avatar movie"}
]

In [38]:
response = chain.batch(questions)

In [39]:
response

['"The Godfather" is a classic American crime film directed by Francis Ford Coppola, released in 1972. It is widely regarded as one of the greatest films ever made and is based on Mario Puzo\'s best-selling novel of the same name. The film stars Marlon Brando, Al Pacino, James Caan, Robert Duvall, and Diane Keaton, among others.\n\n### Plot Overview\nThe story revolves around the Corleone family, a powerful Mafia dynasty in New York City. The film begins with the wedding of Connie Corleone (Talia Shire), the youngest daughter of Don Vito Corleone (Marlon Brando), the head of the family. The wedding is a pivotal moment where various characters are introduced, including Vito\'s sons: Sonny (James Caan), Fredo (John Cazale), and Michael (Al Pacino).\n\nThe narrative then shifts to the attempted assassination of Vito Corleone, which leads to Michael, initially reluctant to join the family business, taking over the reins of the family after his father is wounded. Michael\'s transformation f

In [40]:
chain = RunnablePassthrough() | prompt | llm | output_parser

In [41]:
chain.invoke({"topic":"movies", "question":"Tell me about The Godfather movie"})

'"The Godfather" is a classic American crime film directed by Francis Ford Coppola, released in 1972. It is widely regarded as one of the greatest films ever made and is based on Mario Puzo\'s best-selling novel of the same name. The film stars Marlon Brando, Al Pacino, James Caan, Robert Duvall, and Diane Keaton, among others.\n\n### Plot Overview\nThe story revolves around the Corleone family, a powerful Mafia dynasty in New York City. The film begins with the wedding of Connie Corleone (Talia Shire), the youngest daughter of Don Vito Corleone (Marlon Brando), the head of the family. The wedding is a pivotal moment where various characters are introduced, including Vito\'s sons: Sonny (James Caan), Fredo (John Cazale), and Michael (Al Pacino).\n\nThe narrative then shifts to the attempted assassination of Vito Corleone, which leads to Michael, initially reluctant to join the family business, taking over the reins of the family after his father is wounded. Michael\'s transformation fr

In [42]:
new_chain = RunnablePassthrough() | RunnablePassthrough.assign(topic=lambda x: "movies") | prompt | llm | output_parser

In [43]:
test_chain = RunnablePassthrough() | RunnablePassthrough.assign(topic=lambda x: "movies")

In [44]:
test_chain.invoke({"question":"Tell me about Interstellar movie"})

{'question': 'Tell me about Interstellar movie', 'topic': 'movies'}

In [45]:
new_chain.invoke({"question":"Tell me about Interstellar movie"})

'"Interstellar" is a science fiction film directed by Christopher Nolan, released in 2014. The movie is known for its ambitious storytelling, complex themes, and groundbreaking visual effects. Here are some key points about "Interstellar":\n\n### Plot Summary\nThe story is set in a future where Earth is on the brink of environmental collapse. The protagonist, Cooper (played by Matthew McConaughey), is a widowed engineer and former NASA pilot who is tasked with leading a mission to find a new home for humanity. The mission involves traveling through a wormhole near Saturn to explore potentially habitable planets in a distant galaxy.\n\n### Key Characters\n- **Cooper**: A widowed engineer and former NASA pilot who is the central character of the film.\n- **Murph**: Cooper\'s daughter, who plays a crucial role in the story and has a unique connection with her father.\n- **Brand**: A scientist and astronaut who accompanies Cooper on the mission.\n- **TARS**: A robotic companion with a dry 

### Chain of chains

In [52]:
title = (
    ChatPromptTemplate.from_template(
        "Generate an impactful title for {input}"
    )
    | ChatMistralAI(model="mistral-small-2501", temperature=0)
    | StrOutputParser()
    | {"title" : RunnablePassthrough()}
)

In [53]:
outline = (
    ChatPromptTemplate.from_template(
        "Generate a detailed outline for {title}"
    )
    | ChatMistralAI(model="mistral-small-2501", temperature=0)
    | StrOutputParser()
    | {"outline" : RunnablePassthrough()}
)

In [54]:
blog = (
    ChatPromptTemplate.from_template(
        "Generate 200 word blog post based on the outline: {outline}"
    )
    | ChatMistralAI(model="mistral-small-2501", temperature=0)
    | StrOutputParser()
    | {"blog" : RunnablePassthrough()}
)

In [55]:
summary = (
    ChatPromptTemplate.from_template(
        "Generate a summary for the post {blog}"
    )
    | ChatMistralAI(model="mistral-small-2501", temperature=0)
    | StrOutputParser()
)

In [56]:
content_chain = title | outline | blog | summary

In [57]:
content_chain.invoke({"input":"The impact of AI on jobs"})

"### AI at Work: The Job Market Revolution\n\nArtificial Intelligence (AI) is no longer a futuristic concept; it's a present-day reality transforming industries and reshaping the job market. This article delves into how AI is revolutionizing employment, creating new opportunities and challenges.\n\n#### Understanding AI and Automation\n\nAI encompasses various types, from narrow AI, designed for specific tasks, to general AI, capable of performing any intellectual task a human can, and superintelligent AI, which surpasses human intelligence. AI and automation work through machine learning, natural language processing, and robotics. In the workplace, AI is already making waves in customer service, data analysis, and manufacturing.\n\n#### The Impact of AI on Employment\n\nAI's impact on employment is multifaceted. While it displaces jobs in industries like manufacturing and retail, it also creates new roles in data science, AI development, and cybersecurity. Job descriptions are evolvin